## ReMECS -- Real-time Multimodal Emotion Classification Syatem

## Using multimodal data stream [EEG+EDA+RESP_BELT] 
## Using 1D-CNN + SGD + One pass + Weighted Majority Voting

## Modifications:
- New Feature Set for EDA and Resp Belt. 
- Feature normalization is done.
- RECS with other modalities (EDA + Resp Belt) is incorporated for Real-time Multimodal Emotion Classification System (ReMECS).
- Weighted Majority Vote is used for Decision Fusion. 


## Date: 26 August 2021 at 10:10 AM

In [7]:
#============================
# Import important libraries
#============================
import pandas as pd 
import numpy as np
import tensorflow as tf
import math
import scipy
import pywt
from creme import metrics
import time
import datetime
from statistics import mode
from scipy import stats
from sklearn import preprocessing
from collections import defaultdict, Counter
from scipy.special import expit
from mlxtend.plotting import plot_confusion_matrix
import matplotlib.pyplot as plt

from window_slider import Slider

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
%matplotlib inline

import warnings
warnings.filterwarnings("ignore")

In [8]:
def calculate_entropy(list_values):
    counter_values = Counter(list_values).most_common()
    probabilities = [elem[1]/len(list_values) for elem in counter_values]
    entropy=scipy.stats.entropy(probabilities)
    return entropy

def calculate_statistics(list_values):
    n5 = np.nanpercentile(list_values, 5)
    n25 = np.nanpercentile(list_values, 25)
    n75 = np.nanpercentile(list_values, 75)
    n95 = np.nanpercentile(list_values, 95)
    median = np.nanpercentile(list_values, 50)
    mean = np.nanmean(list_values)
    std = np.nanstd(list_values)
    var = np.nanvar(list_values)
    rms = np.nanmean(np.sqrt(list_values**2))
    return [n5, n25, n75, n95, median, mean, std, var, rms]

def calculate_crossings(list_values):
    zero_crossing_indices = np.nonzero(np.diff(np.array(list_values)> 0))[0]
    no_zero_crossings = len(zero_crossing_indices)
    mean_crossing_indices = np.nonzero(np.diff(np.array(list_values) > np.nanmean(list_values)))[0]
    no_mean_crossings = len(mean_crossing_indices)
    return [no_zero_crossings, no_mean_crossings]

def get_features(list_values):    
    list_values = list_values[0,:]
    entropy = calculate_entropy(list_values)
    crossings = calculate_crossings(list_values)
    statistics = calculate_statistics(list_values)
    return [entropy] + crossings + statistics

In [3]:
#======================================================
# EDA Feature Extraction (Wavelet Features)
#======================================================
def extract_eda_features(raw_eda):
    features =[]
    EDA = raw_eda
    list_coeff = pywt.wavedec(EDA, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features


In [4]:
#======================================================
# RESP BELT Feature Extraction (Wavelet Features)
#======================================================

def extract_resp_belt_features(raw_data):
    features =[]
    resp_belt = raw_data
    list_coeff = pywt.wavedec(resp_belt, 'db4', level=3)
    
#     print(list_coeff)
    for coeff in list_coeff:
        features += get_features(coeff)
    return features

In [5]:
def eeg_features(raw_data):
    ch = 0
    features= []
    def calculate_entropy(list_values):
        counter_values = Counter(list_values).most_common()
        probabilities = [elem[1]/len(list_values) for elem in counter_values]
        entropy=scipy.stats.entropy(probabilities)
        return entropy

    def calculate_statistics(list_values):
        median = np.nanpercentile(list_values, 50)
        mean = np.nanmean(list_values)
        std = np.nanstd(list_values)
        var = np.nanvar(list_values)
        rms = np.nanmean(np.sqrt(list_values**2))
        return [median, mean, std, var, rms]

    def get_features(list_values):    
    #     list_values = list_values[0,:]
        entropy = calculate_entropy(list_values)
        statistics = calculate_statistics(list_values)
        return [entropy] + statistics
    
    for i in range(raw_data.shape[0]):
        ch_data = raw_data[i]
        list_coeff = pywt.wavedec(ch_data, 'db4', level=5)
        for coeff in list_coeff:
            features += get_features(coeff)
            
        ch = ch+1
    return features

In [6]:
##===================================================
# EEG data read from files
##===================================================
def eeg_data(p,v):
    file_eeg = '/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/'+str(p)+'_data_DEAP'+'.csv'
    print(file_eeg)
    df = pd.read_csv(file_eeg,sep=',', header = None, engine='python')
    eeg_sig = df.loc[df.iloc[:,1] == v]
    return eeg_sig

##===================================================
# EDA data read from files
##===================================================
def eda_data(p,v):
    file_eda = '/home/gp/Desktop/MER_arin/DEAP_data/eda_data/'+str(p)+'_GSR_data_from_DEAP.csv'
    print(file_eda)
    df = pd.read_csv(file_eda,sep=',', header = None, engine='python')
    eda_sig = df.loc[df.iloc[:,1] == v]
    return eda_sig

##===================================================
# Resp data read from files
##===================================================
def resp_data(p,v):
    file_resp = '/home/gp/Desktop/MER_arin/DEAP_data/resp_data/'+str(p)+'_Respiration_data_from_DEAP.csv'
    print(file_resp)
    df = pd.read_csv(file_resp,sep=',', header = None, engine='python')
    resp_sig = df.loc[df.iloc[:,1] == v]
    return resp_sig

In [15]:
#==================================
# 1D-CNN model
#==================================
def create_model(x):
    model = Sequential()
    model.add(Conv1D(filters=1, kernel_size=1, activation='relu', input_shape=(1,x.shape[1])))
    model.add(Conv1D(filters=1, kernel_size=1, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=1))
    model.add(Flatten())
    model.add(Dense(10, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))

    opt = tf.keras.optimizers.SGD(learning_rate=0.05)


    model.compile(loss='binary_crossentropy',
                  optimizer=opt,
                  metrics=['accuracy'])
    
    
    print(model.summary())
    
    return model

In [16]:
#=======================================
# MAIN PROGRAM STARTS HERE
#=======================================

segment_in_sec = 10 #in sec
bucket_size = int((8064/60)*segment_in_sec)  #8064 is for 60 sec record
overlap_count = 0

num_classifiers = 3 #Total number of classifiers
w_val =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
w_aro =np.ones(num_classifiers)/num_classifiers #Weights for valence classifiers
beta = 0.5

eta = 0.05
b =1 


optimizer= 'SGD' #optimizer
classifier = '1DCNN_'+str(optimizer)
c = 0
run = 1

participant = 32
videos = 40

global eeg_emotion, eda_emotion, resp_emotion, mer_emotion, all_eta
eeg_emotion = []
eda_emotion = []
resp_emotion = []
mer_emotion = []

all_eta =[]
init_m = 0

#================================================
# Performance matric declaration here
#================================================
mer_acc_val = metrics.Accuracy() #Accuracy
mer_f1m_val = metrics.F1() #F1 measure  
mer_acc_aro = metrics.Accuracy() #Accuracy
mer_f1m_aro = metrics.F1() #F1 measure
mer_roc_val = metrics.ROCAUC()
mer_roc_aro = metrics.ROCAUC() 
mer_mcc_val = metrics.MCC()
mer_mcc_aro = metrics.MCC()


eeg_acc_val = metrics.Accuracy() #Accuracy
eeg_f1m_val = metrics.F1() #F1 measure  
eeg_acc_aro = metrics.Accuracy() #Accuracy
eeg_f1m_aro = metrics.F1() #F1 measure
eeg_roc_val = metrics.ROCAUC()
eeg_roc_aro = metrics.ROCAUC()
eeg_mcc_val = metrics.MCC()
eeg_mcc_aro = metrics.MCC()

eda_acc_val = metrics.Accuracy() #Accuracy
eda_f1m_val = metrics.F1() #F1 measure  
eda_acc_aro = metrics.Accuracy() #Accuracy
eda_f1m_aro = metrics.F1() #F1 measure
eda_roc_val = metrics.ROCAUC()
eda_roc_aro = metrics.ROCAUC()
eda_mcc_val = metrics.MCC()
eda_mcc_aro = metrics.MCC()

resp_acc_val = metrics.Accuracy() #Accuracy
resp_f1m_val = metrics.F1() #F1 measure  
resp_acc_aro = metrics.Accuracy() #Accuracy
resp_f1m_aro = metrics.F1() #F1 measure
resp_roc_val = metrics.ROCAUC()
resp_roc_aro = metrics.ROCAUC()
resp_mcc_val = metrics.MCC()
resp_mcc_aro = metrics.MCC()

 
mer_cm_val  = metrics.ConfusionMatrix()
mer_cm_aro  = metrics.ConfusionMatrix()

eeg_cm_val  = metrics.ConfusionMatrix()
eeg_cm_aro  = metrics.ConfusionMatrix()

eda_cm_val  = metrics.ConfusionMatrix()
eda_cm_aro  = metrics.ConfusionMatrix()
  
resp_cm_val  = metrics.ConfusionMatrix()
resp_cm_aro  = metrics.ConfusionMatrix()
    
itr = 0 #controls the learning rate


for ii in range(0,participant):


        # if(ii >2):
        # 	break

        p =ii+1
        for jj in range(0,videos):
            v = jj+1
            p_v = 'Person:'+ ' ' +str(p)+ ' ' +'Video:'+str(v)
            
            emotion_label =[]
            
            print('------------------------------------------------')
            ##===================================================
            # Data read from files
            ##===================================================
            eeg_sig = eeg_data(p,v)
            eda_sig = eda_data(p,v)
            resp_sig = resp_data(p,v)
            
            #=================================================
            #emotion labels (valence, arousal) mapping 0-1
            #=================================================
            val = eeg_sig.iloc[0,8067]
            aro = eeg_sig.iloc[0,8068]
            
            #valence emotion maping 0-> low valence and 1-> high valence

            if (val >5):
                vl = 1 #high valence
            else:
                vl = 0 #low valence

            #arousal emotion maping 0-> low arousal and 1-> high high arousal
            if (aro >5):
                al = 1 #high arousal
            else:
                al = 0 #low arousal
                
            y_act_val = np.array([vl])
            y_act_aro = np.array([al]) 
            
            
            #=========================================
            # Sliding window starts here 
            #=========================================
            slider_eeg = Slider(bucket_size,overlap_count)
            slider_eda = Slider(bucket_size,overlap_count)
            slider_resp = Slider(bucket_size,overlap_count)
            
            eeg_sig = np.array(eeg_sig.iloc[range(0,32),range(3,8067)]) #keeping only eeg signals
            eda_sig = np.array(eda_sig.iloc[:,range(3,8067)]) #keeping only eda signals
            resp_sig = np.array(resp_sig.iloc[:,range(3,8067)]) #keeping only resp signals
            
            slider_eeg.fit(eeg_sig)
            slider_eda.fit(eda_sig)
            slider_resp.fit(resp_sig)

            while True:
                window_data_eeg = slider_eeg.slide()
                window_data_eda = slider_eda.slide() 
                window_data_resp = slider_resp.slide() 
                
                #=================================================
                # Feature extraction from EEG
                #=================================================
                features_eeg = eeg_features(window_data_eeg)
                eeg = np.array([features_eeg])  #EEG raw feature vector
                x_eeg = preprocessing.normalize(eeg) # EEG normalized features [0,1] 
                
                
                #=================================================
                # Feature extraction from EDA
                #=================================================
                eda_features = extract_eda_features(np.array(window_data_eda))
                eda = np.array([eda_features]) #EDA raw feature vector
                x_eda = preprocessing.normalize(eda) #EDA normalized features
                
                #=================================================
                # Feature extraction from Resp belt
                #=================================================

                resp_features = extract_resp_belt_features(np.array(window_data_resp))
                resp = np.array([resp_features]) #RESP BELT raw feature vector
                x_resp = preprocessing.normalize(resp) #RESP BELT normalized features
            
            
                #===================================================
                # Model initialization
                #===================================================
                if init_m == 0:
                    print('EEG Feature shape{}:'.format(x_eeg.shape))
                    print('EDA Feature shape{}:'.format(x_eda.shape))
                    print('RESP BELT Feature shape{}:'.format(x_resp.shape))

                    #========================
                    # For EEG data model
                    #========================
                    eeg_model_val = create_model(x_eeg)

                    eeg_model_aro = create_model(x_eeg)

                    #========================
                    # For EDA data model
                    #========================
                    eda_model_val = create_model(x_eda)

                    eda_model_aro = create_model(x_eda)

                    #==============================
                    # For Resp Belt data Model
                    #==============================
                    resp_model_val = create_model(x_resp)

                    resp_model_aro = create_model(x_resp)

                    init_m = init_m+1


                #===============================================================
                # Emotion Classification --> Valence and Arousal
                #===============================================================

                if c == 0: #For the first time model will return 0 or None
                    y_pred_val_eeg = np.array([0])
                    y_pred_aro_eeg = np.array([0])
                    
                    y_pred_val_eda = np.array([0])
                    y_pred_aro_eda = np.array([0])
                    
                    y_pred_val_resp = np.array([0])
                    y_pred_aro_resp = np.array([0])
                    
                    eeg_model_val.fit(np.array([x_eeg]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    
                    resp_model_val.fit(np.array([x_resp]) , y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    resp_model_aro.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    
                    c = c+1
                    
                else:
                    #Test the model first 
                    y_pred_val_eeg = eeg_model_val.predict(np.array([x_eeg]))
                    y_pred_aro_eeg = eeg_model_aro.predict(np.array([x_eeg]))
                    
                    y_pred_val_eda = eda_model_val.predict(np.array([x_eda]))
                    y_pred_aro_eda = eda_model_aro.predict(np.array([x_eda]))
                    
                    y_pred_val_resp = resp_model_val.predict(np.array([x_resp]))
                    y_pred_aro_resp = resp_model_aro.predict(np.array([x_resp]))
                    
                    #Binary output scaling 1 if x > 0.5 else 0
                    y_pred_val_eeg = [1 if y_pred_val_eeg > 0.5 else 0]
                    y_pred_aro_eeg = [1 if y_pred_aro_eeg > 0.5 else 0]
                    
                    y_pred_val_eda = [1 if y_pred_val_eda > 0.5 else 0]
                    y_pred_aro_eda = [1 if y_pred_aro_eda > 0.5 else 0]
                    
                    y_pred_val_resp = [1 if y_pred_val_resp > 0.5 else 0]
                    y_pred_aro_resp = [1 if y_pred_aro_resp > 0.5 else 0]
                    
                    #Fit the model once
                    eeg_model_val.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eeg_model_aro.fit(np.array([x_eeg]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    
                    eda_model_val.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    eda_model_aro.fit(np.array([x_eda]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    
                    resp_model_val.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    resp_model_aro.fit(np.array([x_resp]), y_act_val, epochs = 1, batch_size = 1, verbose=0)
                    
                    

                #===========================================
                # Performance matric update
                #===========================================
                
                #Valence classification EEG

                eeg_acc_val = eeg_acc_val.update(y_act_val, y_pred_val_eeg)  # update the accuracy metric

                eeg_f1m_val = eeg_f1m_val.update(y_act_val, y_pred_val_eeg) #update f1 measure metric
                
                eeg_roc_val = eeg_roc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                eeg_mcc_val = eeg_mcc_val.update(y_act_val[0], y_pred_val_eeg[0])
                
                #Arousal classification EEG 

                eeg_acc_aro = eeg_acc_aro.update(y_act_aro, y_pred_aro_eeg)  # update the accuracy metric

                eeg_f1m_aro = eeg_f1m_aro.update(y_act_aro, y_pred_aro_eeg) #update f1 measure metric
                
                eeg_roc_aro = eeg_roc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
                
                eeg_mcc_aro = eeg_mcc_aro.update(y_act_aro[0], y_pred_aro_eeg[0])


                #Valence classification EDA

                eda_acc_val = eda_acc_val.update(y_act_val, y_pred_val_eda)  # update the accuracy metric

                eda_f1m_val = eda_f1m_val.update(y_act_val, y_pred_val_eda) #update f1 measure metric
                
                eda_roc_val = eda_roc_val.update(y_act_val[0], y_pred_val_eda[0])
                
                eda_mcc_val = eda_mcc_val.update(y_act_val[0], y_pred_val_eda[0])
                

                #Arousal classification EDA
                
                eda_acc_aro = eda_acc_aro.update(y_act_aro, y_pred_aro_eda)  # update the accuracy metric

                eda_f1m_aro = eda_f1m_aro.update(y_act_aro, y_pred_aro_eda) #update f1 measure metric
                
                eda_roc_aro = eda_roc_aro.update(y_act_aro[0], y_pred_aro_eda[0])
                
                eda_mcc_aro = eda_mcc_aro.update(y_act_aro[0], y_pred_aro_eda[0])

                #Valence classification Resp Belt

                resp_acc_val = resp_acc_val.update(y_act_val, y_pred_val_resp)  # update the accuracy metric

                resp_f1m_val = resp_f1m_val.update(y_act_val, y_pred_val_resp) #update f1 measure metric
                
                resp_roc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                
                resp_mcc_val = resp_roc_val.update(y_act_val[0], y_pred_val_resp[0])
                

                #Arousal classification Resp Belt

                resp_acc_aro = resp_acc_aro.update(y_act_aro, y_pred_aro_resp)  # update the accuracy metric

                resp_f1m_aro = resp_f1m_aro.update(y_act_aro, y_pred_aro_resp) #update f1 measure metric
                
                resp_roc_aro = resp_roc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                
                resp_mcc_aro = resp_mcc_aro.update(y_act_aro[0], y_pred_aro_resp[0])
                
                
                if slider_eeg.reached_end_of_list(): break
                
            
            ##=============================
            # Confusion Matric Calculation
            ##=============================
            
            eeg_cm_val = eeg_cm_val.update(y_act_val[0], y_pred_val_eeg[0])
            eeg_cm_aro = eeg_cm_aro.update(y_act_aro[0], y_pred_aro_eeg[0])
            
            eda_cm_val = eda_cm_val.update(y_act_val[0], y_pred_val_eda[0])
            eda_cm_aro = eda_cm_aro.update(y_act_aro[0], y_pred_aro_eda[0])
            
            resp_cm_val = resp_cm_val.update(y_act_val[0], y_pred_val_resp[0])
            resp_cm_varo = resp_cm_aro.update(y_act_aro[0], y_pred_aro_resp[0])            
            
            #=============================================================
            # Storing All Results
            #=============================================================
            
            emotion_label.append([y_pred_val_eeg,y_pred_aro_eeg]) #appending valence & arousal predicted EEG
            emotion_label.append([y_pred_val_eda, y_pred_aro_eda]) #appending valence & arousal predicted EDA
            emotion_label.append([y_pred_val_resp, y_pred_aro_resp]) #appending valence & arousal predicted resp

            emotion_label = np.array(emotion_label)
            
                    
            #==============================================================
            # Decision label ensemble --> Weighted Majority Voting
            #==============================================================
            val_label = emotion_label[:,0]
            aro_label = emotion_label[:,1]
            
            #------------------------------------------
            # Valence Class ensemble
            #------------------------------------------
            
            p_val = np.dot([w_val],val_label)
            y_prdt_mer_val = (p_val -0.5)
            if(y_prdt_mer_val > 0):
                mer_val = 1
            else:
                mer_val = 0

            for i in range(val_label.shape[0]):
                if(val_label[i] != y_act_val):
                    w_val[i] = beta*w_val[i]
            
            w_val_sum = sum(w_val) #total sum of weights
            
            w_val = np.array(w_val/w_val_sum) #weight rescaling
            
            #------------------------------------------
            # Arousal Class ensemble
            #------------------------------------------            
            p_val = np.dot([w_aro],aro_label)
            y_prdt_mer_aro = (p_val-0.5)
            if(y_prdt_mer_aro > 0):
                mer_aro = 1
            else:
                mer_aro = 0

            for i in range(aro_label.shape[0]):
                if(aro_label[i] != y_act_aro):
                    w_aro[i] = beta*w_aro[i]
            
            
            w_aro_sum = sum(w_aro) #total sum of weights
            
            w_aro = np.array(w_aro/w_aro_sum) #weight rescaling
            
            #========================================================
            # ReMECS performance metric 
            #========================================================
            mer_acc_val = mer_acc_val.update(y_act_val[0], mer_val)
            mer_f1m_val = mer_f1m_val.update(y_act_val[0], mer_val)
            mer_roc_val = mer_roc_val.update(y_act_val[0], mer_val)
            mer_mcc_val = mer_mcc_val.update(y_act_val[0], mer_val)
            mer_cm_val  = mer_cm_val.update(y_act_val[0], mer_val)
            
            mer_acc_aro = mer_acc_aro.update(y_act_aro[0], mer_aro)
            mer_f1m_aro = mer_f1m_aro.update(y_act_aro[0], mer_aro)
            mer_roc_aro = mer_roc_aro.update(y_act_aro[0], mer_aro)
            mer_mcc_aro = mer_mcc_aro.update(y_act_aro[0], mer_aro)
            mer_cm_aro  = mer_cm_aro.update(y_act_aro[0], mer_aro) 
            
            
            eeg_emotion.append(np.array([p,v,eeg_acc_val.get(), eeg_f1m_val.get(), eeg_roc_val.get(),eeg_mcc_val.get(),
                                         eeg_acc_aro.get(), eeg_f1m_aro.get(),eeg_roc_aro.get(), eeg_mcc_aro.get(), 
                                         y_act_val, y_pred_val_eeg, y_act_aro, y_pred_aro_eeg]))
            
            eda_emotion.append(np.array([p,v,eda_acc_val.get(), eda_f1m_val.get(), eda_roc_val.get(),eda_mcc_val.get(),
                                         eda_acc_aro.get(), eda_f1m_aro.get(), eda_roc_aro.get(), eda_mcc_aro.get(),
                                         y_act_val, y_pred_val_eda, y_act_aro, y_pred_aro_eda]))
            
            resp_emotion.append(np.array([p,v, resp_acc_val.get(), resp_f1m_val.get(), resp_roc_val.get(), resp_mcc_val.get(),
                                          resp_acc_aro.get(),resp_f1m_aro.get(), resp_roc_aro.get(), resp_mcc_aro.get(), 
                                          y_act_val, y_pred_val_resp, y_act_aro, y_pred_aro_resp]))
            
            mer_emotion.append(np.array([p,v,mer_acc_val.get(), mer_f1m_val.get(), mer_roc_val.get(), mer_mcc_val.get(),
                                         mer_acc_aro.get(), mer_f1m_aro.get(), mer_roc_aro.get(), mer_mcc_aro.get(), 
                                         y_act_val, mer_val, y_act_aro, mer_aro]))
            print('===============================================================')
            print(p_v)
            print('Actual emotion [valence: {}, arousal: {}]'.format(y_act_val, y_act_aro))
            print('Single modal Predicted emotion')
            print('[valence, arousal]:')
            print(emotion_label)
            
            print('MER Predicted emotion [valence: {}, arousal: {}]'.format(mer_val, mer_aro))            
            print('..................................................')
            print('EEG')

            print("Valence Accuracy:{}".format(round(eeg_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(eeg_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(eeg_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(eeg_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(eeg_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(eeg_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(eeg_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(eeg_mcc_aro.get(),4)))

            print('..................................................')
            print('EDA')

            print("Valence Accuracy:{}".format(round(eda_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(eda_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(eda_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(eda_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(eda_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(eda_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(eda_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(eda_mcc_aro.get(),4)))


            print('..................................................')
            print('RESP BELT')

            print("Valence Accuracy:{}".format(round(resp_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(resp_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(resp_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(resp_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(resp_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(resp_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(resp_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(resp_mcc_aro.get(),4)))

            print('..................................................')
            print('MER model')

            print("Valence Accuracy:{}".format(round(mer_acc_val.get(),4)))
            print("Valence F1 score:{}".format(round(mer_f1m_val.get(),4)))
            print("Valence ROC score:{}".format(round(mer_roc_val.get(),4)))
            print("Valence MCC score:{}".format(round(mer_mcc_val.get(),4)))
            
            print("Arousal Accuracy:{}".format(round(mer_acc_aro.get(),4)))
            print("Arousal F1 score:{}".format(round(mer_f1m_aro.get(),4)))
            print("Arousal ROC score:{}".format(round(mer_roc_aro.get(),4)))
            print("Arousal MCC score:{}".format(round(mer_mcc_aro.get(),4)))
            



------------------------------------------------
/home/gp/Desktop/MER_arin/DEAP_data/eeg_data/1_data_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
EEG Feature shape(1, 1152):
EDA Feature shape(1, 48):
RESP BELT Feature shape(1, 48):
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_4 (Conv1D)            (None, 1, 1)              1153      
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 1, 1)              2         
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1)              0         
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 1, 1)              0         
_______________________________________

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:2
Actual emotion [valence: [1], arousal: [1]]
Single modal Predicted emotion
[valence, arousal]:
[[[1]
  [1]]

 [[1]
  [1]]

 [[1]
  [1]]]
MER Predicted emotion [valence: 1, arousal: 1]
..................................................
EEG
Valence Accuracy:0.9167
Valence F1 score:0.9565
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:0.9167
Arousal F1 score:0.9565
Arousal ROC score:-0.0
Arousal MCC score:0.0
..................................................
EDA
Valence Accuracy:0.9167
Valence F1 score:0.9565
Valence ROC score:-0.0
Valence MCC score:0.0
Arousal Accuracy:0.9167
Arousal F1 score:0.9565
Arousal ROC score:-0.0
Arousal MCC score:0.0
..................................................
RESP BELT
Valence Accuracy:0.9167
Valence F1 score:0.9565
Valence ROC score:-0.0
Valence MCC score:-0.0
Arousal Accu

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:8
Actual emotion [valence: [1], arousal: [0]]
Single modal Predicted emotion
[valence, arousal]:
[[[1]
  [1]]

 [[1]
  [1]]

 [[1]
  [1]]]
MER Predicted emotion [valence: 1, arousal: 1]
..................................................
EEG
Valence Accuracy:0.8542
Valence F1 score:0.9213
Valence ROC score:0.4881
Valence MCC score:-0.0551
Arousal Accuracy:0.4792
Arousal F1 score:0.6479
Arousal ROC score:0.4792
Arousal MCC score:-0.1459
..................................................
EDA
Valence Accuracy:0.8542
Valence F1 score:0.9213
Valence ROC score:0.4881
Valence MCC score:-0.0551
Arousal Accuracy:0.4792
Arousal F1 score:0.6479
Arousal ROC score:0.4792
Arousal MCC score:-0.1459
..................................................
RESP BELT
Valence Accuracy:0.8542
Valence F1 score:0.9213
Valence ROC score:0.4881
Valence M

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:14
Actual emotion [valence: [0], arousal: [0]]
Single modal Predicted emotion
[valence, arousal]:
[[[0]
  [0]]

 [[0]
  [0]]

 [[0]
  [0]]]
MER Predicted emotion [valence: 0, arousal: 0]
..................................................
EEG
Valence Accuracy:0.7024
Valence F1 score:0.8226
Valence ROC score:0.7024
Valence MCC score:0.4837
Arousal Accuracy:0.4643
Arousal F1 score:0.6306
Arousal ROC score:0.6125
Arousal MCC score:0.253
..................................................
EDA
Valence Accuracy:0.631
Valence F1 score:0.7721
Valence ROC score:0.631
Valence MCC score:0.3621
Arousal Accuracy:0.4167
Arousal F1 score:0.5857
Arousal ROC score:0.5792
Arousal MCC score:0.1978
..................................................
RESP BELT
Valence Accuracy:0.6429
Valence F1 score:0.7808
Valence ROC score:0.6429
Valence MCC sco

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/1_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/1_Respiration_data_from_DEAP.csv
Person: 1 Video:20
Actual emotion [valence: [1], arousal: [1]]
Single modal Predicted emotion
[valence, arousal]:
[[[1]
  [1]]

 [[1]
  [1]]

 [[1]
  [1]]]
MER Predicted emotion [valence: 1, arousal: 1]
..................................................
EEG
Valence Accuracy:0.6333
Valence F1 score:0.6665
Valence ROC score:0.6263
Valence MCC score:0.2548
Arousal Accuracy:0.35
Arousal F1 score:0.389
Arousal ROC score:0.35
Arousal MCC score:-0.3043
..................................................
EDA
Valence Accuracy:0.575
Valence F1 score:0.6267
Valence ROC score:0.5631
Valence MCC score:0.1297
Arousal Accuracy:0.325
Arousal F1 score:0.3793
Arousal ROC score:0.325
Arousal MCC score:-0.3615
..................................................
RESP BELT
Valence Accuracy:0.575
Valence F1 score:0.6202
Valence ROC score:0.5648
Valence MCC score:0

...

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/32_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/32_Respiration_data_from_DEAP.csv
Person: 32 Video:16
Actual emotion [valence: [0], arousal: [1]]
Single modal Predicted emotion
[valence, arousal]:
[[[1]
  [1]]

 [[1]
  [1]]

 [[1]
  [1]]]
MER Predicted emotion [valence: 1, arousal: 1]
..................................................
EEG
Valence Accuracy:0.6277
Valence F1 score:0.6801
Valence ROC score:0.6129
Valence MCC score:0.2339
Arousal Accuracy:0.4244
Arousal F1 score:0.4827
Arousal ROC score:0.4017
Arousal MCC score:-0.2031
..................................................
EDA
Valence Accuracy:0.6265
Valence F1 score:0.6799
Valence ROC score:0.6114
Valence MCC score:0.2311
Arousal Accuracy:0.424
Arousal F1 score:0.4826
Arousal ROC score:0.4012
Arousal MCC score:-0.2042
..................................................
RESP BELT
Valence Accuracy:0.6265
Valence F1 score:0.6798
Valence ROC score:0.6114
Valence 

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/32_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/32_Respiration_data_from_DEAP.csv
Person: 32 Video:22
Actual emotion [valence: [0], arousal: [1]]
Single modal Predicted emotion
[valence, arousal]:
[[[1]
  [1]]

 [[1]
  [1]]

 [[1]
  [1]]]
MER Predicted emotion [valence: 1, arousal: 1]
..................................................
EEG
Valence Accuracy:0.6278
Valence F1 score:0.6808
Valence ROC score:0.6127
Valence MCC score:0.2338
Arousal Accuracy:0.4263
Arousal F1 score:0.4851
Arousal ROC score:0.403
Arousal MCC score:-0.2006
..................................................
EDA
Valence Accuracy:0.6267
Valence F1 score:0.6806
Valence ROC score:0.6112
Valence MCC score:0.2311
Arousal Accuracy:0.4259
Arousal F1 score:0.485
Arousal ROC score:0.4025
Arousal MCC score:-0.2017
..................................................
RESP BELT
Valence Accuracy:0.6267
Valence F1 score:0.6805
Valence ROC score:0.6113
Valence M

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/32_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/32_Respiration_data_from_DEAP.csv
Person: 32 Video:28
Actual emotion [valence: [1], arousal: [0]]
Single modal Predicted emotion
[valence, arousal]:
[[[1]
  [1]]

 [[1]
  [1]]

 [[1]
  [1]]]
MER Predicted emotion [valence: 1, arousal: 1]
..................................................
EEG
Valence Accuracy:0.6271
Valence F1 score:0.6807
Valence ROC score:0.6119
Valence MCC score:0.2323
Arousal Accuracy:0.4276
Arousal F1 score:0.4869
Arousal ROC score:0.4039
Arousal MCC score:-0.1989
..................................................
EDA
Valence Accuracy:0.6259
Valence F1 score:0.6805
Valence ROC score:0.6104
Valence MCC score:0.2295
Arousal Accuracy:0.4272
Arousal F1 score:0.4867
Arousal ROC score:0.4034
Arousal MCC score:-0.2
..................................................
RESP BELT
Valence Accuracy:0.6259
Valence F1 score:0.6804
Valence ROC score:0.6104
Valence MC

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/32_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/32_Respiration_data_from_DEAP.csv
Person: 32 Video:34
Actual emotion [valence: [0], arousal: [1]]
Single modal Predicted emotion
[valence, arousal]:
[[[0]
  [0]]

 [[0]
  [0]]

 [[0]
  [0]]]
MER Predicted emotion [valence: 0, arousal: 0]
..................................................
EEG
Valence Accuracy:0.6269
Valence F1 score:0.6806
Valence ROC score:0.6124
Valence MCC score:0.2334
Arousal Accuracy:0.4283
Arousal F1 score:0.4872
Arousal ROC score:0.4044
Arousal MCC score:-0.1977
..................................................
EDA
Valence Accuracy:0.6257
Valence F1 score:0.6804
Valence ROC score:0.6109
Valence MCC score:0.2307
Arousal Accuracy:0.4279
Arousal F1 score:0.487
Arousal ROC score:0.4039
Arousal MCC score:-0.1988
..................................................
RESP BELT
Valence Accuracy:0.6257
Valence F1 score:0.6803
Valence ROC score:0.611
Valence M

/home/gp/Desktop/MER_arin/DEAP_data/eda_data/32_GSR_data_from_DEAP.csv
/home/gp/Desktop/MER_arin/DEAP_data/resp_data/32_Respiration_data_from_DEAP.csv
Person: 32 Video:40
Actual emotion [valence: [1], arousal: [0]]
Single modal Predicted emotion
[valence, arousal]:
[[[0]
  [0]]

 [[0]
  [0]]

 [[0]
  [0]]]
MER Predicted emotion [valence: 0, arousal: 0]
..................................................
EEG
Valence Accuracy:0.6279
Valence F1 score:0.6807
Valence ROC score:0.6142
Valence MCC score:0.2366
Arousal Accuracy:0.4271
Arousal F1 score:0.4849
Arousal ROC score:0.4032
Arousal MCC score:-0.1997
..................................................
EDA
Valence Accuracy:0.6267
Valence F1 score:0.6805
Valence ROC score:0.6127
Valence MCC score:0.234
Arousal Accuracy:0.4267
Arousal F1 score:0.4848
Arousal ROC score:0.4027
Arousal MCC score:-0.2008
..................................................
RESP BELT
Valence Accuracy:0.6267
Valence F1 score:0.6804
Valence ROC score:0.6127
Valence 

In [17]:
print('EEG Val CM:')
print(eeg_cm_val)

print('EEG aro CM:')
print(eeg_cm_aro)

print('EDA Val CM:')
print(eda_cm_val)

print('EDA aro CM:')
print(eda_cm_aro)

print('RESP Val CM:')
print(resp_cm_val)
print('RESP Aro CM:')
print(resp_cm_aro)

print('ReMECS Val CM:')
print(mer_cm_val)

print('ReMECS aro CM:')
print(mer_cm_aro)

EEG Val CM:
             0      1
      0    297    275
      1    164    544
EEG aro CM:
             0      1
      0    140    403
      1    318    419
EDA Val CM:
             0      1
      0    295    277
      1    162    546
EDA aro CM:
             0      1
      0    139    404
      1    318    419
RESP Val CM:
             0      1
      0    296    276
      1    162    546
RESP Aro CM:
             0      1
      0    139    404
      1    318    419
ReMECS Val CM:
             0      1
      0    296    276
      1    163    545
ReMECS aro CM:
             0      1
      0    139    404
      1    318    419


In [ ]:
print('[acc_val,f1m_val,roc_val,mcc_val,acc_aro,f1m_aro,roc_aro,mcc_aro]')
#=============================
# EEG
#=============================
t_eeg_emotion =pd.DataFrame(eeg_emotion) 
t_eeg_emotion = t_eeg_emotion.iloc[:,range(2,10)]
t_eeg_emotion_avg = np.array(np.mean(t_eeg_emotion))
t_eeg_emotion_std = np.array(np.std(t_eeg_emotion))

print('EEG result Average:', t_eeg_emotion_avg)
print('EEG result std:', t_eeg_emotion_std)

#=============================
# EDA
#=============================
t_eda_emotion =pd.DataFrame(eda_emotion) 
t_eda_emotion = t_eda_emotion.iloc[:,range(2,10)]
t_eda_emotion_avg = np.array(np.mean(t_eda_emotion))
t_eda_emotion_std = np.array(np.std(t_eda_emotion))

print('EDA result Average:', t_eda_emotion_avg)
print('EDA result std:', t_eda_emotion_std)
#=============================
# RESP BELT
#=============================
t_resp_emotion =pd.DataFrame(resp_emotion) 
t_resp_emotion = t_resp_emotion.iloc[:,range(2,10)]
t_resp_emotion_avg = np.array(np.mean(t_resp_emotion))
t_resp_emotion_std = np.array(np.std(t_resp_emotion))

print('Resp Belt result Average:', t_resp_emotion_avg)
print('Resp Belt result std:', t_resp_emotion_std)
#=============================
# MER
#=============================
t_mer_emotion =pd.DataFrame(mer_emotion) 
t_mer_emotion = t_mer_emotion.iloc[:,range(2,10)]
t_mer_emotion_avg = np.array(np.mean(t_mer_emotion))
t_mer_emotion_std = np.array(np.std(t_mer_emotion))

print('MER result Average:', t_mer_emotion_avg)
print('MER result std:', t_mer_emotion_std)

In [18]:
classifier = '1D_CNN_'+str(optimizer)

In [19]:
time_stamp = datetime.datetime.now()
run =1
algo= '1DCNN/' 
filename = "/home/gp/Desktop/MER_arin/12DEAP_RPWE/Results_RPWE_DEAP/"+algo
fname_eeg = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EEG'+'_' +classifier+'_results.csv'
fname_eda = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_EDA'+'_' +classifier+'_results.csv'
fname_resp = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_RESP_BELT'+'_' +classifier+'_results.csv'
fname_mer = filename + 'Run_'+str(run)+'_'+str(time_stamp)+'_all_person_MER'+'_' +classifier+'_results.csv'

column_names = ['Person', 'Video', 'Acc_val', 'F1_val','roc_val', 'mcc_val','Acc_aro',
                'F1_aro', 'roc_aro', 'mcc_aro', 'y_act_val', 'y_pred_val', 'y_act_aro', 'y_pred_aro']

eeg_emotion = pd.DataFrame(eeg_emotion,columns = column_names)
eda_emotion = pd.DataFrame(eda_emotion,columns = column_names)
resp_emotion = pd.DataFrame(resp_emotion,columns = column_names)
mer_emotion = pd.DataFrame(mer_emotion,columns = column_names)

eeg_emotion.to_csv(fname_eeg)
eda_emotion.to_csv(fname_eda)
resp_emotion.to_csv(fname_resp)
mer_emotion.to_csv(fname_mer)


